# Flights Gap Analysis

## Staging

In [4]:
import numpy as np
import pandas as pd
import os
from plotnine import *
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

# Train-Test-Validation Files
train_data_path = r'D:\Summer Practicum\1_Data\training_airlines.csv'
#test_data_path = r'D:\Summer Practicum\1_Data\testing.csv'
#validation_data_path = r'D:\Summer Practicum\1_Data\validation.csv'

df_train = pd.read_csv(train_data_path, low_memory=False)
rdw_df = df_train.copy()
rdw_df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2022,2,4,20,3,2022-04-20,WN,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2022,4,12,25,7,2022-12-25,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2022,4,11,1,2,2022-11-01,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2022,3,9,27,2,2022-09-27,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2022,4,10,13,4,2022-10-13,WN,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


# Recurring Delays Windows

## Departures

In [5]:
# create df for all departures with actual delay (no early arrivals)
rdw_depts = rdw_df[rdw_df["DepDelay"]>0].copy()

# calculate the mean delay to compare later
avg_dept_delay = rdw_df["DepDelay"].mean()

# prep data
day_map = {1: 'Mon',
           2: 'Tue',
           3: 'Wed',
           4: 'Thu',
           5: 'Fri',
           6: 'Sat',
           7: 'Sun'
           }
rdw_depts['DayOfWeek'] = rdw_depts['DayOfWeek'].map(day_map)
group_cols = ['Operating_Airline ', 'DayOfWeek', 'DepTimeBlk']
outbound_grouped = (
    rdw_depts
    .groupby(group_cols)['DepDelay']
    .mean()
    .reset_index(name='DelayCount')
)

# set directory for saving data
os.makedirs("heatmaps/arrivals", exist_ok=True)
os.makedirs("heatmaps/departures", exist_ok=True)

# heatmap by timeblock
airlines = outbound_grouped['Operating_Airline '].unique()
for airline in airlines:
    data = outbound_grouped[outbound_grouped['Operating_Airline '] == airline]
    pivot_df = pd.pivot(data=data, index='DepTimeBlk', columns='DayOfWeek', values='DelayCount')
    
    # Sort columns and rows
    pivot_df = pivot_df.reindex(columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    pivot_df = pivot_df.sort_index()

    # generate plot
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(pivot_df, cmap='Reds', vmin=0, vmax=120, linewidths=0.3, annot=False)
    plt.title(f"Average Departure Delays for {airline}")

    # Add tick at the weekly average
    cbar = ax.collections[0].colorbar
    cbar.ax.axhline(avg_dept_delay, color='black', linestyle='-', linewidth=2)
    cbar.ax.text(1.05, avg_dept_delay, f'Avg: {avg_dept_delay:.1f} min',
                color='black', va='center', ha='left', fontsize=9, transform=cbar.ax.transData)
    
    # save data and close
    filename = f"heatmaps/departures/{airline}_delay_data.csv"
    data.to_csv(filename, index=False)
    hmfilename = f"heatmaps/departures/{airline}_heatmap.png"
    plt.savefig(hmfilename, dpi=300)
    plt.close()


## Arrivals

In [6]:
# create df for all arrivals with actual delay (no early arrivals)
rdw_arrs = rdw_df[rdw_df["ArrDelay"]>0].copy()

# calculate the mean delay to compare later
avg_arr_delay = rdw_df["ArrDelay"].mean()

# prep data
day_map = {1: 'Mon',
           2: 'Tue',
           3: 'Wed',
           4: 'Thu',
           5: 'Fri',
           6: 'Sat',
           7: 'Sun'
           }
rdw_arrs['DayOfWeek'] = rdw_arrs['DayOfWeek'].map(day_map)
group_cols = ['Operating_Airline ', 'DayOfWeek', 'ArrTimeBlk']
outbound_grouped = (
    rdw_arrs
    .groupby(group_cols)['ArrDelay']
    .mean()
    .reset_index(name='DelayCount')
)

# heatmap by timeblock
airlines = outbound_grouped['Operating_Airline '].unique()
for airline in airlines:
    data = outbound_grouped[outbound_grouped['Operating_Airline '] == airline]
    pivot_df = pd.pivot(data=data, index='ArrTimeBlk', columns='DayOfWeek', values='DelayCount')
    
    # Sort columns and rows
    pivot_df = pivot_df.reindex(columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    pivot_df = pivot_df.sort_index()

    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(pivot_df, cmap='Reds', vmin=0, vmax=120, linewidths=0.3, annot=False)
    plt.title(f"Average Arrivals Delays for {airline}")
    

    # Add tick at the weekly average
    cbar = ax.collections[0].colorbar
    cbar.ax.axhline(avg_arr_delay, color='black', linestyle='-', linewidth=2)
    cbar.ax.text(1.05, avg_arr_delay, f'Avg: {avg_arr_delay:.1f} min',
                color='black', va='center', ha='left', fontsize=9, transform=cbar.ax.transData)

    # save data and close
    filename = f"heatmaps/arrivals/{airline}_delay_data.csv"
    data.to_csv(filename, index=False)
    hmfilename = f"heatmaps/arrivals/{airline}_heatmap.png"
    plt.savefig(hmfilename, dpi=300)
    plt.close()

# Arrival Delays by Flight Route

## Data Staging

### Internal

In [ ]:
# create a tuple for all airports origin/destination pairs (we don't mind which direction the flight is heading)
rdw_df['AIRPORT_PAIR'] = rdw_df.apply(lambda x: '-'.join(sorted([x['Origin'], x['Dest']])), axis=1)

In [1]:
# create aggregated dataframe for the map
agg_df = rdw_df[rdw_df['ArrDelay'] > 0].groupby('AIRPORT_PAIR').agg(
    avg_delay=('ArrDelay', 'mean'),
    median_delay=('ArrDelay', 'median'),
    total_delay=('ArrDelay', 'sum'),
    num_flights=('ArrDelay', 'count'),
).reset_index()

# Optional: filter for routes with a significant number of flights
agg_df = agg_df[agg_df['num_flights'] >= 10]


# Get one DistanceGroup per AIRPORT_PAIR
pair_to_group = rdw_df[['AIRPORT_PAIR', 'DistanceGroup']].drop_duplicates(subset='AIRPORT_PAIR') # Get first occurrence - I checked and there's no variability at all so it's safe
agg_df = agg_df.merge(pair_to_group, on='AIRPORT_PAIR', how='left') # Merge with agg_df

# Normalize delay by distance, using distance groups (every 250 miles) 
    # need to consider if we want groups or just miles
agg_df['norm_delay'] = agg_df['avg_delay'] / agg_df['DistanceGroup'].astype(float)

# Optional: filter for n-worst routes to avoid over-cluttering the map
n_filter = 10
top_df = agg_df.sort_values('norm_delay', ascending=False).head(n_filter).copy()

# Normalize values to [0,1] within top-N (combats skewness that messes up the color gradiant)
min_nd = top_df['norm_delay'].min()
max_nd = top_df['norm_delay'].max()
top_df['norm_scaled'] = (top_df['norm_delay'] - min_nd) / (max_nd - min_nd)


NameError: name 'rdw_df' is not defined

### External

In [78]:
# set up column headers because OpenFlights doesn't have them
columns = [
    'AirportID', 'Name', 'City', 'Country',
    'IATA', 'ICAO', 'Latitude', 'Longitude',
    'Altitude', 'Timezone', 'DST', 'TzDatabaseTimeZone',
    'Type', 'Source'
]

# Get coordinates from public OpenFlightd DB and remove duplicates ("N/A" is duplicated, causing indexing errors)
of_airports = pd.read_csv(r'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header=None, names=columns)
of_airports = of_airports[of_airports['IATA'].notna() & (of_airports['IATA'] != '\\N')]

airport_coords = of_airports.set_index('IATA')[['Latitude', 'Longitude']].to_dict('index')
airport_coords['IFP'] = {'Latitude': 35.1574, 'Longitude': -114.5596} # manually add Laughlin/Bullhead, AZ - it is missing from the OpenFlights DB

In [79]:
# split each route into origin and destination coordinates
top_df[['AIRPORT1', 'AIRPORT2']] = top_df['AIRPORT_PAIR'].str.split('-', expand=True)
top_df['LAT1'] = top_df['AIRPORT1'].map(lambda x: airport_coords.get(x, {}).get('Latitude'))
top_df['LON1'] = top_df['AIRPORT1'].map(lambda x: airport_coords.get(x, {}).get('Longitude'))
top_df['LAT2'] = top_df['AIRPORT2'].map(lambda x: airport_coords.get(x, {}).get('Latitude'))
top_df['LON2'] = top_df['AIRPORT2'].map(lambda x: airport_coords.get(x, {}).get('Longitude'))

## Map Generating

In [106]:
import plotly.graph_objects as go
import plotly.io as pio
import random

# Get color from colormap
cmap = plt.get_cmap('YlOrRd')

# Helper function to get line color to work with the scaled Normalized values
def get_rgba(norm_val):
    r, g, b, a = cmap(norm_val)
    return f'rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.8)'


# Helper function to add markers to any flight path for tooltip hovering
def interpolate_coords(lat1, lon1, lat2, lon2, steps=5):
    lats = np.linspace(lat1, lat2, steps)
    lons = np.linspace(lon1, lon2, steps)
    return list(lats), list(lons)



fig = go.Figure() # create map plot
use_start = True  # initialize toggle for label positioning
# Fill map with flight paths (lines)
for _, row in top_df.iterrows():
    cust_color = get_rgba(row['norm_scaled'])

    # Add color-coded trace of flight path
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[row['LON1'], row['LON2']],
        lat=[row['LAT1'], row['LAT2']],
        mode='lines',
        line=dict(width=0.5 + row['avg_delay'] / 30, color=cust_color),
        opacity=0.8,
        name=row['AIRPORT_PAIR'],
        text=f"Route: {row['AIRPORT_PAIR']}<br>Avg Delay: {row['avg_delay']:.1f} min<br>Flights: {row['num_flights']}",
        hoverinfo='text'
    ))

    # Interpolate midpoints for better hover coverage
    interp_lats, interp_lons = interpolate_coords(row['LAT1'], row['LON1'], row['LAT2'], row['LON2'], steps=10)

    # Add invisible markers along the flight path to enable tooltip when hovering over the line
    # (otherwise the tooltip only shows up when hovering around the endpoints)
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=interp_lons,
        lat=interp_lats,
        mode='markers',
        marker=dict(size=3, color=cust_color, opacity=0.001),  # marker indistinguishable from line but still renders - need to see if this causes issues on non-GPU-accelerated machines
        text=f"Route: {row['AIRPORT_PAIR']}<br>Avg Delay: {row['avg_delay']:.1f} min<br>Flights: {row['num_flights']}",
        hoverinfo='text',
        showlegend=False
    ))

    # Alternate label position between origin and destination
    if use_start:
        label_lat, label_lon = row['LAT1'], row['LON1']
    else:
        label_lat, label_lon = row['LAT2'], row['LON2']
    
    # Jitter to prevent label overlapping (will be useful if we increase the N-Filter)
    label_lat += random.uniform(0, 0.5) if use_start else -random.uniform(0, 0.2)

    # Flip toggle
    use_start = not use_start

    # Add label trace
    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[label_lon],
        lat=[label_lat],
        mode='text',
        text=[row['AIRPORT_PAIR']],
        textfont=dict(size=12, color='black'),
        showlegend=False,
        hoverinfo='skip'
    ))

# Create the plot's layout
fig.update_layout(
    title=dict(
        text=f'Flight Routes with Worst Normalized Delays (per 250 Miles)<br>Worst {n_filter} Routes with >=10 Flights',
        x=0.5,  # Center the title
        xanchor='center',
        font=dict(size=18)
    ),
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
    ),
    showlegend=True
)

fig.show()

# Save the interactive map to an HTML file
pio.write_html(fig, file="worst_flight_delays_map.html", auto_open=False)
